In [ ]:
from dolfin import *
from multiphenics import *
import matplotlib.pyplot as plt
from dolfin import plot


def eps(u):
    #return sym(nabla_grad(u))
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

I = Identity(2)

u_degree = 2
p_degree = 1

n = 20
ratio = 4.0
interf = 2.0

interf_id = 1
top_id = 2
bottom_id = 3
right_id = 4
left_id = 5
mesh = RectangleMesh(Point(0.0), Point(ratio, 1.0), int(n*ratio), n)

fluidmarker = MeshFunction("size_t", mesh, mesh.topology().dim(), 0)
fluid = CompiledSubDomain("x[0] <= interf + DOLFIN_EPS", interf=interf)
fluid.mark(fluidmarker, 1)

porousmarker = MeshFunction("size_t", mesh, mesh.topology().dim(), 0)
porous = CompiledSubDomain("x[0] >= interf - DOLFIN_EPS", interf=interf)
porous.mark(porousmarker, 1)

bottom = CompiledSubDomain("on_boundary && near(x[1], 0.0)")
top = CompiledSubDomain("on_boundary && near(x[1], 1.0)")
left = CompiledSubDomain("near(x[0], 0.0)")
right = CompiledSubDomain("on_boundary && near(x[0], end)", end=ratio)
interface = CompiledSubDomain("near(x[0], interf, DOLFIN_EPS)", interf=interf)


boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() -1, 0)
bottom.mark(boundaries, bottom_id)
top.mark(boundaries, top_id)
right.mark(boundaries, right_id)
left.mark(boundaries, left_id)
interface.mark(boundaries, interf_id)

subdomains = MeshFunction("size_t", mesh, mesh.topology().dim(), 0)
porous.mark(subdomains, 1)
fluid.mark(subdomains, 2)



fluidmesh = MeshRestriction(mesh, fluid)
porousmesh = MeshRestriction(mesh, porous)
Sig = MeshRestriction(mesh, interface)


n = FacetNormal(mesh)("+")  # 
n = Constant([1.0, 0.0])
t = as_vector((-n[1], n[0]))

dxF = Measure("dx", domain=mesh, subdomain_data=fluidmarker)(1)
dxP = Measure("dx", domain=mesh, subdomain_data=porousmarker)(1)
dS = Measure("dS", domain=mesh, subdomain_data=boundaries)
ds = Measure("ds", domain=mesh, subdomain_data=boundaries)

ds_Sig = dS(interf_id)

areaP = assemble(1.0*dxP)
areaF = assemble(1.0*dxF)
lengthI = assemble(1.*dS(interf_id))
length_bottom = assemble(1.*ds(bottom_id))

V = VectorFunctionSpace(mesh, "CG", u_degree)
W = FunctionSpace(mesh, "CG", p_degree)

H = BlockFunctionSpace([V, W, V, W, W],
                        restrict=[fluidmesh, fluidmesh,
                                  porousmesh, porousmesh, porousmesh])

trial = BlockTrialFunction(H)
u, pF, d, pP, phi = block_split(trial)
test = BlockTestFunction(H)
v, qF, w, qP, psi = block_split(test)

# define constants

E = 1e5               # Young modulus
nu = 0.48             # Poisson ratio

A = 1e-4
rho_s = Constant(1e3)
mu_s = Constant(E/(2.0*(1.0+nu)))

rho_f = Constant(1e3)
mu_f = Constant(1e-3)

lmbda = Constant(nu*E/((1.0-2.0*nu)*(1.0+nu)))
alpha = Constant(1.0)
kappa = Constant(1e-8)

#C = Constant(1.0)
gamma = Constant(0.5)

# define sources and forces

g = Constant([0.0, 0.0])
f = Constant([0.0, 0.0])

# define boundary conditions (fluid left and porous right)

inflow = DirichletBC(H.sub(0), Expression(("A*sin(x[1]*pi)", "0.0"), A=A, degree=2), left)

no_slip_top = DirichletBC(H.sub(0), Constant([0.0, 0.0]), top)
no_slip_bottom = DirichletBC(H.sub(0), Constant([0.0, 0.0]), bottom)

fluid_bcs = [inflow, no_slip_bottom, no_slip_top]


outflow_por = DirichletBC(H.sub(3), Constant(0.0), right)
fix_top = DirichletBC(H.sub(2), Constant([0.0, 0.0]), top)
fix_bottom = DirichletBC(H.sub(2), Constant([0.0, 0.0]), bottom)
fix_right = DirichletBC(H.sub(2), Constant([0.0, 0.0]), right)


porous_bcs = [fix_bottom, fix_top,fix_right, outflow_por]

bcs = BlockDirichletBC(fluid_bcs + porous_bcs)


# define forms
def a_F(u,v):
    return 2*mu_f*inner(eps(u), eps(v))*dxF \
            + (gamma*mu_f/sqrt(kappa))*inner(u("+"), t)*inner(v("+"),t)*ds_Sig
            
def c_F(u,w,v):
    return rho_f * dot(dot(u, nabla_grad(w)), v)*dxF

def b_1_F(v, qF):
    return - qF*div(v)*dxF

def b_2_Sig(v, qP):
    return qP("+")*inner(v("+"), n)*ds_Sig

def b_3_Sig(v, d):
    return - ((gamma*mu_f/sqrt(kappa))*inner(v("+"),t)*inner(d("+"), t)*ds_Sig)

def b_4_Sig(w,qP):
    return -qP("+") * dot(w("+"),n)*ds_Sig

def a_1_P(d, w):
    return 2.0*mu_s*inner(eps(d), eps(w))*dxP

def b_1_P(w, psi):
    return - psi*div(w)*dxP

def a_2_P(pP,qP):
    return (kappa/mu_f) *inner(grad(pP), grad(qP))*dxP

def b_2_P(psi, qP):
    return (alpha/lmbda)*psi*qP*dxP

def a_3_P(phi, psi):
    return (1.0/lmbda)*phi*psi*dxP

def F_F(v):
    return rho_f *dot(g, v)*dxF

def F_P(w):
    return rho_s*inner(f, w)*dxP

def G(qP):
    return rho_f*inner(g, grad(qP))*dxP \
            - rho_f*inner(g, n)*qP("+")*ds_Sig 

# define system:

rhs = [F_F(v), 0, F_P(w), G(qP), 0]

# order trial: u, pF, d, pP, phi
# order test: v, qF, w, qP, psi


lhs = [[ a_F(u,v)       , b_1_F(v, pF), 0            , b_2_Sig(v, pP), 0                ],
       [ b_1_F(u, qF)   ,  0          , 0            , 0             , 0                ],
       [ b_3_Sig(u, w)  ,  0          , a_1_P(d,w)   , b_4_Sig(w, pP), b_1_P(w, phi)    ],
       [ b_2_Sig(u, qP),  0          , 0            ,  a_2_P(pP, qP) , 0                ], # sign of b_2_Sig(u, qP)!
       [ 0              ,  0          , b_1_P(d, psi), b_2_P(psi, pP), -a_3_P(phi, psi) ]]

AA = block_assemble(lhs, keep_diagonal=True)
FF = block_assemble(rhs)
bcs.apply(AA, FF)

sol = BlockFunction(H)
block_solve(AA, sol.block_vector(), FF, "mumps")
u, pF, d, pP, phi = block_split(sol)

names = ["velocity u", "fluid pressure pF", "displacement d",
         "fluid pressure in porous domain pP", "total pressure phi"]
funcs = [u, pF, d, pP, phi]
[f.rename(names[i], "") for i,f in enumerate(funcs)]

output = XDMFFile("../results/stokes_biot.xdmf")
output.parameters["rewrite_function_mesh"] = False
output.parameters["functions_share_mesh"] = True
for f in funcs:
    output.write(f, 0.0)
output.close()


In [ ]:
# check interface conditions

dx = Measure("dx", domain=mesh, subdomain_data=subdomains)
dS = Measure("dS", domain=mesh, subdomain_data=boundaries, subdomain_id=1)

ds_Sig = dS(interf_id)
# continuity of normal flux:

normal_flux_L = assemble(dot(u, n)*ds_Sig)
normal_flux_R = assemble(((kappa/mu_f)* dot(-grad(pP)("-") , n)*ds_Sig + Constant(0.0)*dx))

print(f"normal fluxes: {normal_flux_L}, {normal_flux_R}")

In [ ]:
# momentum conservation_
mom_con_L0 = assemble(((2*mu_f*eps(u("+")) - pF*I)*n)[0]*ds_Sig + Constant(0.0)*dx)
mom_con_L1 = assemble(((2*mu_f*eps(u("+")) - pF*I)*n)[1]*ds_Sig + Constant(0.0)*dx)

mom_con_R0 = assemble(((2*mu_s*eps(d("-")) - phi*I)* n)[0]*ds_Sig + Constant(0.0)*dx)
mom_con_R1 = assemble(((2*mu_s*eps(d("-")) - phi*I)* n)[1]*ds_Sig + Constant(0.0)*dx)

print(f"mom con 0: {mom_con_L0}, {mom_con_R0}")
print(f"mom con 1: {mom_con_L1}, {mom_con_R1}")

In [ ]:
# fluid normal stresses
fns_L = assemble(- dot(n, (2*mu_f*eps(u("+")) - pF*I)*n)*ds_Sig + Constant(0.0)*dx)
fns_R = assemble(pP*ds_Sig)
print(f"fluid normal stresses: {fns_L} =? {fns_R}")

In [ ]:
for i, func in enumerate(funcs):
    plt.figure(figsize=(15,4))
    c = plot(func)
    plt.title(func.name())
    plt.colorbar(c)